# Solving Sudoku with exact cover

Exact cover algorithm implemented by QUBO in quantum annealer

In [1]:
import numpy as np
import time
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dwave.samplers import SimulatedAnnealingSampler
import dwave.inspector

## Some functions

In [80]:
def count_zeros(sudoku):
    count=0;
    for y in range(size2):
        for x in range(size2):
            if sudoku[y][x]==0:
                count += 1
    return count

def print_sudoku(sudoku):
    for y in range(size2):
        if y!=0 and y%ssize==0:
            if ssize==4: print('----+----+----+----')      
            if ssize==3: print('---+---+---')      
            if ssize==2: print('--+--')      
        for x in range(size2):
            if x!=0 and x%ssize==0:
                print('|',end='')
            n = int(sudoku[y][x])
            print('.' if n==0 else n , end='')
        print('')
        
def sudoku_merge(sudoku1, sudoku2):
    sudoku_res = np.zeros((ssize*ssize,ssize*ssize))
    for y in range(size2):
        for x in range(size2):
            if sudoku1[y][x]==0:
                sudoku_res[y][x] = sudoku2[y][x]
            else:
                sudoku_res[y][x] = sudoku1[y][x]
                if sudoku2[y][x]>0:
                    print('! Solver has filled a cell with constant number: {},{}'.format(y+1,x+1))
    return sudoku_res

## Load sudoku

sudoku file structure: first line is number of sudokus. First line of every sudoku is the size of sudoku.

3_level_48.ss: These should be rather easy sudokus, 48 cells not filled.

In [81]:
f = open('testdata/3_level_48.ss','r')
#f = open('testdata/2_level_varia.ss','r')
scount = int(f.readline())
ssize = int(f.readline())

sudoku=np.zeros((scount,ssize*ssize,ssize*ssize))

for i in range(scount):
    for j in range(ssize):        
        for k in range(ssize):
            line = f.readline()
            for l in range(ssize*ssize):
                c = line[l+int(l/ssize)]
                sudoku[i][j*ssize+k][l] = 0 if c=='.' else int(c)
        line = f.readline()

## Create set and subset from sudoku

Sudoku can be converted to exact cover problem:
- set elements are all option in all cells. So in empty sudoku there is 9x9x9 elements. To get little bit better efficiency, here only empy cells are counted in.
- subset block: 1) every cell can have only one number, 2) every 3x3 block can have only one number each, 3) every row can have only one number each, 4) every column can have only one number each

Every four subset block has 81 subsets like this: 1) there is 81 cells in sudoku, 2) sudoku has 9 blocks, for wich one subset for each number, 3) 9 rows, for wich one subset for each number, 4) 9 columns, for wich one subset for each number.

Some preprocessing is made, known numbers are one large subset.

In [82]:
sind = 0             # which sudoku from the list
size2 = ssize*ssize

U = [i for i in range(size2*size2*4)]  # 4 blocks of subsets
V = []                                  # Set element names are row*81 + col*9 + "1-9"
cell_id = []

y=0
x=0
while y<size2:
    if sudoku[sind][y][x]==0:
        for j in range(size2):
            cell_id.append((y,x,j+1))
            # cell, block, column, row restriction
            V.append([y*size2+x, 
                      size2*size2 + int(y/ssize)*size2*ssize + int(x/ssize)*size2 + j,
                      2*size2*size2 + y*size2 + j, 
                      3*size2*size2 + x*size2 + j]) 
    x += 1
    if x==size2:
        x = 0
        y += 1

# We have one more large subset, subset of known numbers
V.append([])
ind = len(V)-1
for y in range(size2):
    for x in range(size2):
        if sudoku[sind][y][x]!=0:
            n = sudoku[sind][y][x]-1
            V[ind].append(int(y*size2+x))
            V[ind].append(int(size2*size2 + int(y/ssize)*size2*ssize + int(x/ssize)*size2 + n))
            V[ind].append(int(2*size2*size2 + y*size2 + n))
            V[ind].append(int(3*size2*size2 + x*size2 + n))

z = count_zeros(sudoku[sind])

print('Number of blanks:',z)            
print('Number of elements:',len(U))
print('Number of subsets:',len(V))
print('Number of subsets if no zipping:',size2*size2*size2)

#for v in V:
#    print(v)

Number of blanks: 48
Number of elements: 324
Number of subsets: 433
Number of subsets if no zipping: 729


## Create QUBO

Constraints
- total number of elements in subset should be (len(U))
- each element only in one subset

In [92]:
Q = np.zeros((len(V),len(V)))
t1 = time.time()

# Total elements constraint

max_count = 0

for i in range(len(V)):
    Q[i][i] =- len(V[i])
    max_count += len(V[i])

# each element only in one subset

for a in U:
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            if a in V[i] and a in V[j]:
                Q[i][j] += len(V[i]) + len(V[j]) + 2
t2 = time.time()
print('Time used for construction Q (s): {:.1f}'.format((t2-t1)))
print('Q dimensions:',Q.shape)
print('Max count:',max_count)

# LAST PENALTY -132 # PROBLEM THAT DO NOT CHOOSE LAST ONE

#for y in range(Q.shape[0]):
#    for x in range(Q.shape[1]):
#        print('{:2d} '.format(int(Q[y][x])), end='')
#    print('')

Time used for construction Q (s): 3.3
Q dimensions: (433, 433)
Max count: 1860


## Creat BQM from QUBO

In [93]:
bqm = dimod.BinaryQuadraticModel(Q, 'BINARY')

## Local heuristic classical solver

Local deterministic solver can not be used because: "Maximum allowed dimension exceeded"

In [94]:
t1 = time.time()
sampleset = SimulatedAnnealingSampler().sample(bqm, num_reads=1000)
t2 = time.time()
print('Time used by solver (s): {:.1f}'.format((t2-t1)))
print('Lowest energy reached:',sampleset.first.energy)
print('Lowest energy should be:',-size2*size2*4)   

Time used by solver (s): 12.2
Lowest energy reached: -192.0
Lowest energy should be: -324


## Then analyse results...

In [86]:
sudoku_res = np.zeros((ssize*ssize,ssize*ssize))
r = sampleset.first.sample
for k,v in r.items():
    if v==1 and k<len(cell_id):
        y,x,n = cell_id[k]
        sudoku_res[y][x] = n

In [87]:
print_sudoku(sudoku[sind])

..9|..3|.1.
..1|.27|.8.
...|15.|963
---+---+---
5.8|..1|.49
94.|..6|.3.
...|..9|...
---+---+---
726|3.8|1.4
.34|.6.|..2
...|...|...


In [88]:
print_sudoku(sudoku_res)

21.|75.|6.8
98.|2..|1.3
453|..8|...
---+---+---
.4.|91.|5..
..9|53.|4.7
765|82.|319
---+---+---
...|.7.|.4.
3..|4.9|75.
574|681|932


In [89]:
merge = sudoku_merge(sudoku[sind],sudoku_res)

In [90]:
print_sudoku(merge)

219|753|618
981|227|183
453|158|963
---+---+---
548|911|549
949|536|437
765|829|319
---+---+---
726|378|144
334|469|752
574|681|932


In [76]:
count=0;
for y in range(size2):
    for x in range(size2):
        if merge[y][x]==0:
            count += 1
print('Number of cells not filled:', count)

Number of cells not filled: 0


### Some results

With 9x9
- sudoku 0, num_reads=1.000: 11 seconds, energy -168, not filled 6 (wrong answers)
- sudoku 0, num_reads=10.000: 109 seconds, energy -172, not filled 5 (wrong answers)

With 4x4
- sudoku 0, num_reads=1.000: 1 seconds, energy -64, not filled 0

# Quantum solver

In [14]:
machine = DWaveSampler(solver={'chip_id': 'Advantage_system4.1'})
print('Chip:', machine.properties['chip_id'])
print('Qubits:', machine.properties['num_qubits'])

Chip: Advantage_system4.1
Qubits: 5760


In [15]:
sampleset2 = EmbeddingComposite(machine).sample(bqm, num_reads=1000)

In [16]:
qtime = sampleset2.info['timing']['qpu_access_time'] / 1000
qubits = sum(len(x) for x in sampleset2.info['embedding_context']['embedding'].values())
print('Lowest energy reached:',sampleset.first.energy)
print('Occurences at that level:',sampleset.first.num_occurrences)
print('QPU time used (ms): {:.1f}'.format(qtime))
print('Physical qubits used: {}'.format(qubits))

Lowest energy reached: -56.0
Occurences at that level: 1
QPU time used (ms): 190.2
Physical qubits used: 147


In [17]:
sudoku_res = np.zeros((ssize*ssize,ssize*ssize))
r = sampleset2.first.sample
for k,v in r.items():
    if v==1 and k<len(cell_id):
        y,x,n = cell_id[k]
        sudoku_res[y][x] = n

In [18]:
print_sudoku(sudoku[sind])

..|.3
.4|..
--+--
..|32
..|..


In [19]:
print_sudoku(sudoku_res)

21|..
..|1.
--+--
1.|..
32|41


In [20]:
merge = sudoku_merge(sudoku[sind],sudoku_res)

In [21]:
print_sudoku(merge)

21|.3
.4|1.
--+--
1.|32
32|41


In [22]:
count=0;
for y in range(size2):
    for x in range(size2):
        if merge[y][x]==0:
            count += 1
print('Number of cells not filled:', count)

Number of cells not filled: 4


In [23]:
dwave.inspector.show(sampleset2)

Serving Inspector on http://127.0.0.1:18000/?problemId=b397bd53-3d9e-4b17-8a9e-df5d833349f3

'http://127.0.0.1:18000/?problemId=b397bd53-3d9e-4b17-8a9e-df5d833349f3'